# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
!pip install catboost==1.2.5


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install optuna==3.6.1


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install xgboost==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 2.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import optuna
from optuna.integration import OptunaSearchCV
from sklearn.model_selection import cross_val_score

# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    catboost_params = {
        'iterations': trial.suggest_int('catboost_iterations', 100, 1000),
        'learning_rate': trial.suggest_float('catboost_learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('catboost_depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('catboost_l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('catboost_border_count', 32, 128),
        'loss_function': 'CrossEntropy',
        'random_seed': 42,
        'verbose': 0
    }

    xgboost_params = {
        'n_estimators': trial.suggest_int('xgboost_n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('xgboost_learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('xgboost_max_depth', 4, 10),
        'reg_alpha': trial.suggest_float('xgboost_reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('xgboost_reg_lambda', 0, 10),
        'random_state': 42,
    }
    
    catboost = CatBoostClassifier(**catboost_params)
    xgboost = XGBClassifier(**xgboost_params, use_label_encoder=False, eval_metric='logloss')
    
    ensemble = VotingClassifier(estimators=[
        ('catboost', catboost), 
        ('xgboost', xgboost)
    ], voting='soft')
    
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()), 
        ('ensemble', ensemble)
    ])

    # Using StratifiedKFold for handling imbalanced data
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')
    
    return scores.mean()

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)

# Best parameters
best_params = study.best_params
print("Best parameters:", best_params)

# Extract the best parameters for each model
best_catboost_params = {k.replace('catboost_', ''): v for k, v in best_params.items() if 'catboost_' in k}
best_xgboost_params = {k.replace('xgboost_', ''): v for k, v in best_params.items() if 'xgboost_' in k}

# Train final model with the best parameters
catboost = CatBoostClassifier(**best_catboost_params)
xgboost = XGBClassifier(**best_xgboost_params, use_label_encoder=False, eval_metric='logloss')

ensemble = VotingClassifier(estimators=[
    ('catboost', catboost), 
    ('xgboost', xgboost)
], voting='soft')

pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('ensemble', ensemble)
])
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
pipeline.fit(X_df, y_df)

# Perform cross-validation on all data
scores = cross_val_score(pipeline, X_df, y_df, cv=5)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

# Predict probabilities for submission data
y_submission_proba = pipeline.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict.to_csv('bankChurn_ensemble.csv', index=False)


268:	learn: 0.3053694	total: 24.2s	remaining: 32.7s
269:	learn: 0.3053477	total: 24.3s	remaining: 32.6s
270:	learn: 0.3053319	total: 24.4s	remaining: 32.5s
271:	learn: 0.3053130	total: 24.5s	remaining: 32.4s
272:	learn: 0.3052941	total: 24.6s	remaining: 32.4s
273:	learn: 0.3052784	total: 24.7s	remaining: 32.3s
274:	learn: 0.3052584	total: 24.8s	remaining: 32.2s
275:	learn: 0.3052356	total: 24.9s	remaining: 32.1s
276:	learn: 0.3052058	total: 25s	remaining: 32s
277:	learn: 0.3051848	total: 25.1s	remaining: 32s
278:	learn: 0.3051658	total: 25.2s	remaining: 31.9s
279:	learn: 0.3051509	total: 25.3s	remaining: 31.8s
280:	learn: 0.3051361	total: 25.4s	remaining: 31.7s
281:	learn: 0.3051126	total: 25.5s	remaining: 31.6s
282:	learn: 0.3050975	total: 25.6s	remaining: 31.6s
283:	learn: 0.3050803	total: 25.7s	remaining: 31.5s
284:	learn: 0.3050630	total: 25.8s	remaining: 31.4s
285:	learn: 0.3050470	total: 25.9s	remaining: 31.3s
286:	learn: 0.3050294	total: 26s	remaining: 31.2s
287:	learn: 0.305009

In [ ]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 19.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import optuna
from optuna.integration import OptunaSearchCV
from sklearn.model_selection import cross_val_score

# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'loss_function': 'CrossEntropy',
        'random_seed': 42,
        'verbose': 0
    }
    
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()), 
        ('classifier', CatBoostClassifier(**param))
    ])

    # Using StratifiedKFold for handling imbalanced data
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')
    
    return scores.mean()

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Best parameters
best_params = study.best_params
print("Best parameters:", best_params)

# Train final model with the best parameters
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('classifier', CatBoostClassifier(**best_params))
])
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
pipeline.fit(X_df, y_df)

# Perform cross-validation on all data
scores = cross_val_score(pipeline, X_df, y_df, cv=5)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

# Predict probabilities for submission data
y_submission_proba = pipeline.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict.to_csv('bankChurn_catop.csv', index=False)


[I 2024-05-17 11:38:59,292] A new study created in memory with name: no-name-28aee94a-b05e-4c52-8605-4c39a11b4c1a
[I 2024-05-17 11:59:26,308] Trial 0 finished with value: 0.8965664327292636 and parameters: {'iterations': 835, 'learning_rate': 0.04325210842206607, 'depth': 10, 'l2_leaf_reg': 7.756381301946842, 'border_count': 99}. Best is trial 0 with value: 0.8965664327292636.


# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Model Training

In [ ]:
# Membuat pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('classifier', CatBoostClassifier(random_seed=42, verbose=0))
])


param_grid = {
    'classifier__iterations': [500, 1000, 1500],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__depth': [6, 8, 10],
    'classifier__l2_leaf_reg': [1, 3, 5, 7],
    'classifier__border_count': [32, 64, 128],
    'classifier__loss_function': ['CrossEntropy', 'Logloss']
}


# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# GridSearchCV setup dengan parameter yang lebih luas
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model dengan metric tambahan
y_pred = best_model.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))
print("F1-Score on test set:", f1_score(y_test, y_pred))


Fitting 10 folds for each of 648 candidates, totalling 6480 fits


In [ ]:
# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.76      0.57      0.65      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.79     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
[[24790  1262]
 [ 3004  3951]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        <catboost.core.CatBoostClassifier object at 0x7fa5d55dbaf0>)]),
             param_grid={'classifier__border_count': [64],
                         'classifier__depth': [10],
                         'classifier__iterations': [500],
                         'classifier__l2_leaf_reg': [5],
                         'classifier__learning_rate': [0.01],
                         'classifier__loss_function': ['CrossEntropy']},
             scoring='roc_auc', verbose=1)

In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

[0.01629698 0.85410913 0.02370677 ... 0.02108736 0.19028542 0.16779655]


,id,Exited
0,165034,0.016297
1,165035,0.854109
2,165036,0.023707
3,165037,0.221638
4,165038,0.378906
...,...,...
110018,275052,0.035880
110019,275053,0.096423
110020,275054,0.021087
110021,275055,0.190285


In [ ]:
submission_dict.to_csv('bankChurn_cat2.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>